Our team is interested in tackling the OpenVaccine Kaggle competition (https://www.kaggle.com/c/stanford-covid-vaccine) by developing a neutrual network model to predict COVID-19 mRNA vaccine degradation. Train set provides 2400 RNA sequences, and each sequence has three RNA structure features (structure, reactivity, predicted loop type) and five ground truths that evaluated through experiments (deg_pH10, deg_Mg_pH10, deg_50C, deg_Mg_50C, reactivity) and their correponding errors. The goal of this competiton is to use RNA structural features to predict the degradation rates at each base of RNA sequence at three experimental conditions, including reactivity, deg_Mg_pH10, and deg_Mg_50C.

The additional challenge is problem in this exercise is the different size of training and testing target. We have decided to focus on the public test (see below) to simply the issue. If time permits, we will look to address the bigger sized target prediction.

In [9]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics


# SK-learn library for importing the newsgroup data.

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

from sklearn.model_selection import train_test_split

import nltk

# Tools for counting letters in the sequences
from collections import Counter as count
import plotly.express as px

### Mounting Google Drive


In [10]:
!pip install --upgrade google-api-python-client
!pip install google-colab
from google.colab import drive
drive.mount('/content/drive')

Streaming output truncated to the last 5000 lines.
  copying pandas/plotting/_core.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/plotting
  copying pandas/plotting/_timeseries.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/plotting
  copying pandas/plotting/_compat.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/plotting
  copying pandas/plotting/_misc.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/plotting
  creating build/lib.macosx-10.9-x86_64-3.8/pandas/arrays
  copying pandas/arrays/__init__.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/arrays
  creating build/lib.macosx-10.9-x86_64-3.8/pandas/api
  copying pandas/api/__init__.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/api
  creating build/lib.macosx-10.9-x86_64-3.8/pandas/errors
  copying pandas/errors/__init__.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/errors
  creating build/lib.macosx-10.9-x86_64-3.8/pandas/compat/numpy
  copying pandas/compat/numpy/__init__.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/compat/numpy
  copyin

ModuleNotFoundError: ignored

### Load the train data

In [ ]:
full_train = pd.read_json('/content/drive/MyDrive/1 MIDS/W207 Applied Machine Learning/Final Project/train.json', lines=True)
full_train = full_train.set_index(keys='index')
full_train.info()
full_train.head()

### Load the test data

In [ ]:
full_test = pd.read_json('/content/drive/MyDrive/1 MIDS/W207 Applied Machine Learning/Final Project/test.json', lines=True)
full_test = full_test.set_index(keys='index')
full_test.info()
full_test.head()

## Start EDA here

First, we start with sanity check. According to competition description, the test set is a mixture of 'private test' and 'public test' data, differ by the length of RNA sequence (seq_length). Here we separate the test set into two sets. 

In [ ]:
public_test = full_test[full_test.seq_length==107].reset_index()
private_test = full_test[full_test.seq_length==130].reset_index()
print(public_test.shape)
print(private_test.shape)

Checking the training data as with quick descrption.


In [ ]:
include =['object', 'float', 'int']
descriptive_summary = full_train.describe(include = include)

descriptive_summary

The data is structured quite complicated with different length for sequences can be different in different data set, so just double check that length is expected. 

In [ ]:
def check_seq_length (data,seq, expected_length):
  return data.apply(lambda x:len(x[seq]) == x[expected_length], axis = 1)

for seq in ['sequence', 'structure', 'predicted_loop_type']: 
  print(f'Is the length for {seq} as expected?', all(check_seq_length(full_train, seq, 'seq_length')))


for seq in ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']: 
  print(f'Is the length for {seq} as expected?', all(check_seq_length(full_train, seq, 'seq_scored')))



In [ ]:
# Function to plot correlation matricies
def corrdot(*args, **kwargs):
    corr_r = args[0].corr(args[1], 'pearson')
    corr_text = f"{corr_r:2.2f}".replace("0.", ".")
    ax = plt.gca()
    ax.set_axis_off()
    marker_size = abs(corr_r) * 10000
    ax.scatter([.5], [.5], marker_size, [corr_r], alpha=0.6, cmap="coolwarm",
               vmin=-1, vmax=1, transform=ax.transAxes)
    font_size = abs(corr_r) * 40 + 5
    ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction",
                ha='center', va='center', fontsize=font_size)

RNA contains four bases and form G-C and A-U paris. COVID-19 mRNA vaccine is single-stranded RNA and it forms pairs as it floats and folds itself. Among the two pair types, G-C pair tends to be more statble than A-U pair due to the bonding strength. Therefore, the higher weight of G and C bases in a RNA will make RNA less degradable. Furthermore, structure and loop type also affects RNA degradation. Unpaired bases are more vulnerable to temperature and UV challenges. Stem stucture tend to be more stable tham loops.
Below, we are exploring the weight of each base in each RNA sequence, weight of possible pairings, and weight of different structure types. 

What is the fraction of each base in each observation? 




In [ ]:
# Count the fraction of all the bases in the sequences
bases = []

for base in range(len(full_train)):
    counts = dict(count(full_train.iloc[base]['sequence']))
    bases.append((
        counts['A'] / 107,
        counts['G'] / 107,
        counts['C'] / 107,
        counts['U'] / 107
    ))
    
bases = pd.DataFrame(bases, columns=['A_percent', 'G_percent', 'C_percent', 'U_percent'])
bases

In [ ]:
include =['object', 'float', 'int']
base_summary = bases.describe(include = include)

base_summary

Of the 4 bases, A has the highest average prevalence while U is the least prevalent on average.

What is fraction of each base pair in each observation? 

In [ ]:
pairs = []
all_partners = []
for base in range(len(full_train)):
  partners = [-1 for symbol in range(130)]
  pairs_dict = {('U', 'G'): 0, ('C', 'G'): 0, ('U', 'A'): 0, ('G', 'C'): 0, ('A', 'U'): 0, ('G', 'U'): 0}
  queue = []
  for symbol in range(0, len(full_train.iloc[base]['structure'])):
    if full_train.iloc[base]['structure'][symbol] == '(':
      queue.append(symbol)
    if full_train.iloc[base]['structure'][symbol] == ')':
      first = queue.pop()
      pairs_dict[(full_train.iloc[base]['sequence'][first], full_train.iloc[base]['sequence'][symbol])] += 1
      partners[first] = symbol
      partners[symbol] = first
  
  all_partners.append(partners)
  
  pairs_num = 0
  pairs_unique = [('U', 'G'), ('C', 'G'), ('U', 'A'), ('G', 'C'), ('A', 'U'), ('G', 'U')]
  for item in pairs_dict:
    pairs_num += pairs_dict[item]
  add_tuple = list()
  for item in pairs_unique:
    add_tuple.append(pairs_dict[item]/pairs_num)
  pairs.append(add_tuple)
    
pairs = pd.DataFrame(pairs, columns=['U-G', 'C-G', 'U-A', 'G-C', 'A-U', 'G-U'])
full_train['partners'] = all_partners
pairs

In [ ]:
include =['object', 'float', 'int']
pair_summary = pairs.describe(include = include)

pair_summary

Looking at the base pairs, on average, the G-C pair is the most prevalent while the U-G pair is the least

What are the total counts of base pairs in the whole training set?

In [ ]:
pairs_dict = {('U', 'G'): 0, ('C', 'G'): 0, ('U', 'A'): 0, ('G', 'C'): 0, ('A', 'U'): 0, ('G', 'U'): 0}
queue = []
for base in range(len(full_train)):
  observation = full_train.iloc[base]
  for symbol in range(len(observation['structure'])):
    if observation['structure'][symbol] == '(':
      queue.append(symbol)
    if observation['structure'][symbol] == ')':
      first = queue.pop()
      pairs_dict[(observation['sequence'][first], observation['sequence'][symbol])] += 1

                
pairs_dict

In [ ]:
names = []
values = []
for item in pairs_dict:
    names.append(item)
    values.append(pairs_dict[item])
    
df = pd.DataFrame()
df['pair'] = names
df['count'] = values
df['pair'] = df['pair'].astype(str)

fig = px.bar(
    df, 
    x='pair', 
    y="count", 
    orientation='v', 
    title='Pair types', 
    height=400, 
    width=800
)
fig.show()

In the predicted loop type column, the following symbols were used to represent
base pair categories.

S: paired "Stem"

M: Multiloop

I: Internal loop 

B: Bulge

H: Hairpin loop 

E: dangling End 

X: eXternal loop 

What is the fraction of each category in each observation?

In [ ]:
loops = []
for prediction in range(len(full_train)):
    counts = dict(count(full_train.iloc[prediction]['predicted_loop_type']))
    types = ['E', 'S', 'H', 'B', 'X', 'I', 'M']
    row = []
    for item in types:
      if item in counts:
        row.append(counts[item] / 107)
      else:
        row.append(0)
    loops.append(row)
    
loops = pd.DataFrame(loops, columns=types)
loops

Looking at the loop type summaries, on average, S is the most prevalent type at 44.21% and B is the least present at 1.12%.

In [ ]:
include =['object', 'float', 'int']
loops_summary = loops.describe(include = include)

loops_summary

In [ ]:
res_dict = {'E':0, 'S':0, 'H':0, 'B':0, 'X':0, 'I':0, 'M':0}
for prediction in range(len(full_train)):
    observation = full_train.iloc[prediction]
    pred_symbol = dict(count(observation['predicted_loop_type']))
    for item in pred_symbol:
      if item in pred_symbol:
        res_dict[item] += pred_symbol[item]

res_dict

In [ ]:
names = []
values = []
for item in res_dict:
    names.append(item)
    values.append(res_dict[item])
    
df = pd.DataFrame()
df['loop_type'] = names
df['count'] = values

In [ ]:
fig = px.bar(
    df, 
    x='loop_type', 
    y="count", 
    orientation='v', 
    title='Predicted loop types', 
    height=400, 
    width=600
)
fig.show()

In [ ]:
#pairs and loops
pairs.reset_index(drop=True, inplace=True)
loops.reset_index(drop=True, inplace=True)

correlation_frame = pd.concat([pairs, loops], axis=1)
correlation_frame

g = sns.PairGrid(correlation_frame, aspect=1.4, diag_sharey=False)
g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color': 'black'})
g.map_diag(sns.histplot, kde_kws={'color': 'black'})
g.map_upper(corrdot)

Looking at correlations between base pairs and loop types, we see a high correlation of -.82 between loop types E and S. It looks like there may be an strong grouping of data anchoring this relationship. There is also a strong inverse relationship of -.60 between U-A and G-C pairs.

Signal to noise ratio exploration

In [ ]:
fig = px.histogram(
    full_train, 
    "signal_to_noise", 
    nbins=25, 
    title='signal_to_noise histogram', 
    width=700,
    height=500
)
fig.show()

In [ ]:
ds = full_train['SN_filter'].value_counts().reset_index()
ds.columns = ['SN_filter', 'count']
fig = px.pie(
    ds, 
    values='count', 
    names="SN_filter", 
    title='SN_filter pie chart', 
    width=500, 
    height=500
)
fig.show()

Looking for correlation in the means of the prediction targets showed us correlation among the observations/samples. For example, observations with a higher mean deg_50_Mg_50C had a higher meand deg_ph10. This relationship should be further explored at the individual base level. 



In [ ]:
full_train['mean_reactivity'] = full_train['reactivity'].apply(lambda x: np.mean(x))
full_train['mean_deg_Mg_pH10'] = full_train['deg_Mg_pH10'].apply(lambda x: np.mean(x))
full_train['mean_deg_Mg_50C'] = full_train['deg_Mg_50C'].apply(lambda x: np.mean(x))
full_train['mean_deg_pH10'] = full_train['deg_pH10'].apply(lambda x: np.mean(x))
full_train['mean_deg_50C'] = full_train['deg_50C'].apply(lambda x: np.mean(x))

full_train_plots = full_train.iloc[:,19:25]

full_train_plots

sns.set(style='white', font_scale=1)
g = sns.PairGrid(full_train_plots, aspect=1.4, diag_sharey=False)
g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color': 'black'})
g.map_diag(sns.histplot, kde_kws={'color': 'black'})
g.map_upper(corrdot)

## Start modeling here

### Feature creation

In this problem, our initial input variables are RNA sequence, structure, and loop types, which are represented by characters. The models can not directly consuming those features and we will need to convert them into a numerical format.  

In [ ]:

# Create an dictionary to map character to a integer
token2int = {x:i for i, x in enumerate('().ACGUBEHIMSX')}

def tokenise(data, dic):

  """
  Function to convert sequence, structure and predited_loop_type into one long numerical 
  feature array for each RNA sample 
  """

  # grab the data 
  temp = list(data.sequence) + list(data.structure) +list(data.predicted_loop_type)
  temp = np.array(temp)

  # creating mapping array 
  k = np.array(list(dic.keys()))
  v = np.array(list(dic.values()))

  # Get argsort indices
  sidx = k.argsort()

  # map the initial array to integers 
  ks = k[sidx]
  vs = v[sidx]
  return vs[np.searchsorted(ks,temp)]

# apply the function to train and test data 
full_train['feature_array'] = full_train.apply(lambda x: tokenise(x, token2int), axis = 1)
public_test['feature_array'] =public_test.apply(lambda x: tokenise(x, token2int), axis = 1)

In [ ]:
# Checking if the shape is as expected 
np.array(full_train['feature_array'].tolist()).shape

In [ ]:
 # split into a training, a dev and and a test data
 train_data, dev_test = train_test_split(
    full_train, test_size=.2, random_state=34)
 
dev_data, test_data = train_test_split(
    dev_test, test_size=.5, random_state=34)

dev_data.shape 
train_data.shape

In [ ]:
train_data

In [ ]:
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)


# train_feature = np.array(train_data['feature_array'].tolist())
# train_reactivity = np.array(train_data['reactivity'].tolist())

# dev_feature = np.array(dev_data['feature_array'].tolist())
# dev_reactivity = np.array(dev_data['reactivity'].tolist())


# clf_single = MLPRegressor()
# clf_single.fit(train_feature, train_reactivity)
# print(f'Initial model accuracy {clf_single.score(dev_feature, dev_reactivity)}')


# # def a grid search for var_smoothing 
# params = {'alpha': [1.0e-10, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0],
#           'hidden_layer_sizes': [(100,), (200,), (100, 100), (100, 50, 50, 50)]
#           }

# # start a grid search with min train data 
# clf = GridSearchCV(clf_single, params, cv=5)
# clf.fit(train_feature, train_reactivity)

# # get the best performing model from grid search 
# clf_best = clf.best_estimator_
# clf_best.fit(train_feature, train_reactivity)
# print(f'Final model accuracy {clf_best.score(dev_feature, dev_reactivity)}')


We are choosing to start with neural network due to the complexity of the problem. The input features are just tokenised biological representation, they are not strictly numerical nor categorical. So we think a neural network should be able to handle the complexity inside the problem. General field research and browsing through past submission also proven the choice.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def build_nn(input_feature, output_target, params,  dev_feature, dev_target):
  """
  Training function to conduct grid search for nerual net works 
  """    

  # initiate a single ANN
  clf_single = MLPRegressor(max_iter=300, learning_rate = 'adaptive')
  # clf_single.fit(input_feature, output_target)
  # print(f'Initial model accuracy {clf_single.score(dev_feature, dev_reactivity)}')

  # def a grid search for var_smoothing 

  # start a grid search with 5 fold CV
  clf = GridSearchCV(clf_single, params, cv=5, 
                     n_jobs = 2,
                     scoring = 'neg_root_mean_squared_error')
  clf.fit(input_feature, output_target)

  # get the best performing model from grid search 
  clf_best = clf.best_estimator_
  clf_best.fit(input_feature, output_target)
  
  return clf_best

In [ ]:
# start grid search for all desired outputs
targets = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']
nn_estimator = [] 
predictions = []
scores = []
        
#def a set of params to use for grid search 
params = {'alpha': [ 0.0001, 0.001, 0.01, 0.1, 1.0, 2.0, 10.0],
          'hidden_layer_sizes': [(100,), (200,), (100, 100), (100, 50, 50, 50)]
          }

# prep feature 
train_feature = np.array(train_data['feature_array'].tolist())
dev_feature = np.array(dev_data['feature_array'].tolist())
test_feature = np.array(test_data['feature_array'].tolist())


for target in targets: 
  # prep the data
  output_target = np.array(train_data[target].tolist())
  dev_target = np.array(dev_data[target].tolist())
  test_target = np.array(test_data[target].tolist())


  # start grid search
  best_model = build_nn(train_feature, output_target, params,  dev_feature, dev_target)
  # save the best model
  nn_estimator +=[best_model]
  # and include prediction
  target_predict = best_model.predict(test_feature)
  predictions.append(list(target_predict))

  score = np.sqrt(metrics.mean_squared_error(test_target, target_predict))
  scores.append([score])
  print(f'Final model for {target} has RMSE: {score}')


# It takes realy long time to do the gridsearch, so make some noise when it finish
import IPython
display(IPython.display.Audio(url="https://www.soundhelix.com/examples/mp3/SoundHelix-Song-14.mp3", autoplay=True))

In [ ]:
score = metrics.mean_squared_error(dev_target, target_predict)
scores.append([score])
print(f'Final model for {target} has RMSE: {score}')


In [ ]:
best_model.predict(test_feature)

### Showing the prediction result

In [ ]:
result = pd.DataFrame(dict(zip(targets, predictions)), index=test_data.id)
result.to_csv('/content/drive/MyDrive/1 MIDS/W207 Applied Machine Learning/Final Project/base_line_result.csv')

## Some thoughts on next steps
1. the best performing models in the competition submissions are variants of LSTM, which makes sense as we always want to previous bases in a sequence would matter, should we should take a look
2. the bpps data is worthy exprimenting.
3. the sequence been scored is a shorter set compared to sequence provieded, maybe we can throw away the bottom part to reduce the dimension.
4. more feature engineering, using the base pairs for e.g.
    - explore relationship of base pairs with prediction targets in more granular detail. For example are G bases associated with higher reactivity?
5. Use the competition specific scoring metric to evaluate performance.
6. We used seperate models for 5 output variables, since each output is already non-scalar, we can also try to predict all 5 output with one model.
7. The ANN are suffering from covergence warmings despite a large epoches. We could try other optimisation methods. 